In [5]:
'''
1.給我XGBoost範例，以台灣金融業鷹眼模型為例 
2.並且就之前討論的調參順序，檢視模型正確率是否提升 
3.你說的超參數重點： scale_pos_weight (關鍵） , min_child_weight, gamma 
4.接著整體的 max_depth learning_rate estimators


'''


#以下提供一個 以台灣金融業「鷹眼模型」風格的 XGBoost 範例，並依照你之前提到的調參順序（重點參數 → 整體參數），檢視模型正確率是否提升。

'''

1. 範例場景：台灣金融業鷹眼模型
假設資料集為信用卡交易，目標是 偵測是否為可疑交易（fraud / 非fraud）。特徵包含：
•	客戶年齡、信用卡等級
•	交易金額、交易次數
•	是否跨境、是否高風險行業
目標變數：is_fraud (1=詐欺，0=正常)。


'''
#2. XGBoost 建模流程 (Python)
import xgboost as xgb 
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.metrics import accuracy_score, classification_report

# 分割資料 
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, stratify=y ) 

# Step 1: 初始模型 (baseline) 
xgb_clf = xgb.XGBClassifier( objective="binary:logistic", eval_metric="logloss", random_state=42, use_label_encoder=False ) 
xgb_clf.fit(X_train, y_train) 
y_pred = xgb_clf.predict(X_test) 
print("Baseline Accuracy:", accuracy_score(y_test, y_pred))

'''
#3. 調參順序與重點
(A) 不平衡處理 — scale_pos_weight
金融詐欺通常正負樣本極度不平衡。
計算方式：
'''

scale = float(sum(y == 0)) / sum(y == 1)

'''
(B) 控制過擬合 — min_child_weight, gamma
•	min_child_weight: 每個葉子節點最小樣本權重和，越大越保守
•	gamma: 分裂需要的最小 loss reduction，越大越保守
(C) 整體結構 — max_depth, learning_rate, n_estimators
•	max_depth: 控制樹深度
•	learning_rate: shrinkage，通常 0.05~0.2
•	n_estimators: 樹的數量
'''


#4. GridSearch 調參範例
param_grid = { "scale_pos_weight": [scale], "min_child_weight": [1, 3, 5],"gamma": [0, 0.1, 0.2, 0.5], "max_depth": [3, 5, 7], "learning_rate": [0.05, 0.1, 0.2], "n_estimators": [200, 400, 600] } 

grid = GridSearchCV( estimator=xgb.XGBClassifier( objective="binary:logistic", eval_metric="auc", random_state=42, use_label_encoder=False ),param_grid=param_grid, scoring="accuracy", cv=3, verbose=1, n_jobs=-1 ) 
grid.fit(X_train, y_train)
print("Best params:", grid.best_params_) 
print("Best CV Accuracy:", grid.best_score_)

# 測試集表現 
y_pred_best = grid.best_estimator_.predict(X_test) 
print("Test Accuracy:", accuracy_score(y_test, y_pred_best)) 
print(classification_report(y_test, y_pred_best))

'''
5. 分析正確率提升
•	Baseline 模型：未調參 → Accuracy 可能只有 0.70 左右
•	加入 scale_pos_weight：對抗樣本不平衡 → Recall 提升
•	加入 min_child_weight、gamma：減少過擬合 → Accuracy、F1 Score 提升
•	最後調整 max_depth、learning_rate、n_estimators → 模型更穩定，整體正確率可望提升到 0.80~0.85+




GridSearchCV 的用途是 自動化超參數搜尋與交叉驗證。
核心功能：
1.	遍歷參數組合
o	你提供一個參數網格 param_grid。
o	它會嘗試所有組合，例如 (min_child_weight=1, gamma=0.1, max_depth=5, …)。
2.	交叉驗證 (Cross Validation)
o	每組參數會跑 K 折 (例如 3-fold) 切分資料，避免單次切分的偶然性。
o	輸出平均表現（如 Accuracy、AUC）。
3.	找最佳模型
o	自動回傳表現最佳的參數組合 best_params_。
o	best_estimator_ 直接就是最佳模型，可拿來預測測試集。
與手動調參的差異
•	手動：一次改一個參數 → 慢
•	GridSearchCV：系統化遍歷所有可能組合 → 全面，但耗時
若資料量大，可以改用 RandomizedSearchCV（隨機取樣部分組合），速度快。
'''




NameError: name 'X' is not defined